## Preparation

Execute the following cell once after allocating runtime.

Clones the repo and installs it as an editable package

In [ ]:
!git clone https://github.com/alexeiplatzer/unitree-go2-mjx-rl.git
!pip install -e unitree-go2-mjx-rl/

Cloning into 'unitree-go2-mjx-rl'...
remote: Enumerating objects: 618, done.
remote: Counting objects: 100% (618/618), done.
remote: Compressing objects: 100% (385/385), done.
remote: Total 618 (delta 287), reused 512 (delta 184), pack-reused 0 (from 0)
Receiving objects: 100% (618/618), 22.23 MiB | 10.87 MiB/s, done.
Resolving deltas: 100% (287/287), done.
Obtaining file:///content/unitree-go2-mjx-rl
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 46.9 MB/s eta 0:00:0

### Next steps
After installing the package:  
- Upload the trained policy parameters file in the Colab file browser
- Restart (not delete!) the kernel, to make the installed pacakge available

## Setup GPU rendering

Sets up colab GPUs to be used for rendering. Not needed if using CPU. Must be executed in every session!

In [ ]:
# Set up GPU rendering.
from google.colab import files
import distutils.util
import os
import subprocess
if subprocess.run('nvidia-smi').returncode:
  raise RuntimeError(
      'Cannot communicate with GPU. '
      'Make sure you are using a GPU Colab runtime. '
      'Go to the Runtime menu and select Choose runtime type.')

# Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
# This is usually installed as part of an Nvidia driver package, but the Colab
# kernel doesn't install its driver via APT, and as a result the ICD is missing.
# (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
  with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
    f.write("""{
    "file_format_version" : "1.0.0",
    "ICD" : {
        "library_path" : "libEGL_nvidia.so.0"
    }
}
""")

# Configure MuJoCo to use the EGL rendering backend (requires GPU)
print('Setting environment variable to use GPU rendering:')
%env MUJOCO_GL=egl

Setting environment variable to use GPU rendering:
env: MUJOCO_GL=egl


## Render

Pull the repo in case any changes were pushed. Any updates should be immediately available as the package was installed in editable mode.

In [ ]:
!git -C ./unitree-go2-mjx-rl/ checkout old-python
!git -C ./unitree-go2-mjx-rl/ pull

Already on 'old-python'
Your branch is up to date with 'origin/old-python'.
Already up to date.


Render the policy and save to the Colab content folder. Don't forget to download it!

In [ ]:
from brax.io import model

teacher_params = model.load_params("/content/ppo_teacher_policy")
student_params = model.load_params("/content/ppo_student_policy")
model.save_params(
    "/content/ppo_teacher_student_policy",
    (teacher_params, student_params),
)

FileNotFoundError: [Errno 2] No such file or directory: '/content/ppo_teacher_policy'

In [ ]:
from etils.epath import Path
from quadruped_mjx_rl import render
from quadruped_mjx_rl.environments.ppo_enhanced_legacy import EnhancedEnvironmentConfig

repo_path = Path("/content/unitree-go2-mjx-rl")
configs_path = repo_path / "configs"

render(
    configs_path / "unitree_go2_environment.yaml",
    configs_path / "ppo_simple.yaml",
    configs_path / "render_simple.yaml",
    trained_model_path=Path("/content/ppo_simple_policy"),
    init_scene_path=repo_path / "resources/unitree_go2/scene_mjx.xml",
    animation_save_path="ppo_simple.gif",
    # environment_config=EnhancedEnvironmentConfig(name="go2_teacher"),
)

In [ ]:
!pip show brax

Name: brax
Version: 0.12.1
Summary: A differentiable physics engine written in JAX.
Home-page: http://github.com/google/brax
Author: Brax Authors
Author-email: no-reply@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, dm_env, etils, flask, flask_cors, flax, grpcio, gym, jax, jaxlib, jaxopt, jinja2, ml_collections, mujoco, mujoco-mjx, numpy, optax, orbax-checkpoint, Pillow, pytinyrenderer, scipy, tensorboardX, trimesh, typing-extensions
Required-by: quadruped_mjx_rl
